Import needed libraries

In [1]:
import pandas as pd

from networkx.drawing.nx_agraph import write_dot
import networkx as nx
import numpy as np
import re
import pandasql

Load Contacts_covid_model csv file for the infected location node

In [2]:
ContactDS = pd.read_csv('Data/contacts_covid_model.csv')
ContactDS=ContactDS.rename(columns={'time':'Time','from':'Source','to':'Target','weight':'Weight','label':'Location'})
event_location=ContactDS[['Source','Target','Location']]
event_location = event_location.applymap(str)
#event_location.to_csv("event_location.csv")
event_location

,Source,Target,Location
0,14298,14357,Restaurant
1,425,6831,School
2,301,15510,ConstructionSite
3,1779,18245,Shop
4,9585,18081,School
...,...,...,...
1769380,5452,17652,CovidHospital
1769381,8818,12335,SmallFamily
1769382,8111,18308,SmallFamily
1769383,5706,5708,SmallFamily


Extracting unique location and assigning color to change nodes font

In [3]:
#Assigning background colour, cluster for location
#Location_color = event.Location.unique()
#Location_color = pd.DataFrame(Location_color,columns=['Location'])
#Location_color['bgcolor']=['#800080','#0000FF','#FF00FF','#000080','#008080','#00FFFF','#008000','#00FF00','#808000','#FFFF00','#800000','#000000','#808080','#f77f00','#6a040f','#4ea8de','#e29578','#1b4332','#7209b7']
#Location_color['shape'] = ['polygon','Msquare','box','egg','circle','diamond','septagon','pentagon','house','trapezium','hexagon','octagon','rect','doublecircle','square','Mdiamond','tab','folder','triangle']
#Location_color['Cluster'] = range(1,1+len(Location_color))
#Location_Filter = Location_color.applymap(str)
#Location_Filter

In [4]:
#Assigning background colour, cluster for location
Infect_Location=['Restaurant','School','Shop','Office','ConstructionSite','CovidHospital','Hospital','Nursery','CareHome']
Location_color = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9']
Location_shape = ['circle','Msquare','box','egg','triangle','diamond','septagon','pentagon','house']
Location_Filter = pd.DataFrame([Infect_Location,Location_color,Location_shape],index=['Location','bgcolor','shape']).T
Location_Filter['Cluster'] = range(1,1+len(Location_Filter))
Location_Filter = Location_Filter.applymap(str)
Location_Filter

,Location,bgcolor,shape,Cluster
0,Restaurant,#8dd3c7,circle,1
1,School,#ffffb3,Msquare,2
2,Shop,#bebada,box,3
3,Office,#fb8072,egg,4
4,ConstructionSite,#80b1d3,triangle,5
5,CovidHospital,#fdb462,diamond,6
6,Hospital,#b3de69,septagon,7
7,Nursery,#fccde5,pentagon,8
8,CareHome,#d9d9d9,house,9


In [5]:
event_infect = pd.merge(event_location, Location_Filter, on=['Location', 'Location'], how='inner')
#event_infect.to_csv("InfectionLocation.csv")
event_infect

,Source,Target,Location,bgcolor,shape,Cluster
0,14298,14357,Restaurant,#8dd3c7,circle,1
1,15357,19456,Restaurant,#8dd3c7,circle,1
2,8611,18859,Restaurant,#8dd3c7,circle,1
3,10734,18854,Restaurant,#8dd3c7,circle,1
4,558,15685,Restaurant,#8dd3c7,circle,1
...,...,...,...,...,...,...
1010636,2822,7463,CareHome,#d9d9d9,house,9
1010637,7432,12702,CareHome,#d9d9d9,house,9
1010638,9450,19543,CareHome,#d9d9d9,house,9
1010639,3851,18613,CareHome,#d9d9d9,house,9


In [6]:
#Segregation of Target Node and Cluster
TNL=event_infect[['Target','Location','bgcolor','shape','Cluster']]
TNL.to_csv("InfectionLocation.csv")
TNL = TNL.rename(columns={'Target':'Node'})
TNL = pd.DataFrame.drop_duplicates(TNL)
TNL

,Node,Location,bgcolor,shape,Cluster
0,14357,Restaurant,#8dd3c7,circle,1
1,19456,Restaurant,#8dd3c7,circle,1
2,18859,Restaurant,#8dd3c7,circle,1
3,18854,Restaurant,#8dd3c7,circle,1
4,15685,Restaurant,#8dd3c7,circle,1
...,...,...,...,...,...
990435,1657,CareHome,#d9d9d9,house,9
991245,5532,CareHome,#d9d9d9,house,9
993644,3809,CareHome,#d9d9d9,house,9
995645,1232,CareHome,#d9d9d9,house,9


In [7]:
#Segregation of Source Node and cluster
SNL=event_infect[['Source','Location','bgcolor','shape','Cluster']]
SNL = SNL.rename(columns={'Source':'Node'})
SNL = pd.DataFrame.drop_duplicates(SNL)
SNL

,Node,Location,bgcolor,shape,Cluster
0,14298,Restaurant,#8dd3c7,circle,1
1,15357,Restaurant,#8dd3c7,circle,1
2,8611,Restaurant,#8dd3c7,circle,1
3,10734,Restaurant,#8dd3c7,circle,1
4,558,Restaurant,#8dd3c7,circle,1
...,...,...,...,...,...
992032,19326,CareHome,#d9d9d9,house,9
993451,19608,CareHome,#d9d9d9,house,9
993808,18537,CareHome,#d9d9d9,house,9
995865,18758,CareHome,#d9d9d9,house,9


In [8]:
#Combining Both Source and Target Node with Location and font color
NL = TNL.append(SNL)
NL = NL.drop_duplicates(subset=['Node'], keep='first')
NL.to_csv("InfectionLocation.csv")
NL

,Node,Location,bgcolor,shape,Cluster
0,14357,Restaurant,#8dd3c7,circle,1
1,19456,Restaurant,#8dd3c7,circle,1
2,18859,Restaurant,#8dd3c7,circle,1
3,18854,Restaurant,#8dd3c7,circle,1
4,15685,Restaurant,#8dd3c7,circle,1
...,...,...,...,...,...
686216,125,Shop,#bebada,box,3
985770,1047,CareHome,#d9d9d9,house,9
985771,56,CareHome,#d9d9d9,house,9
985793,295,CareHome,#d9d9d9,house,9


In [9]:
#event_infect = pd.merge(Location_Filter, CL, on=['Location', 'Location'], how='inner')
#event_infect = pd.DataFrame(event_infect)
#event_infect = event_infect.sort_values(by=['Node', 'Cluster']).drop_duplicates(subset=['Node'], keep='first')
#event_infect
#event_infect.to_csv("infectionccccc.csv")

Loading Infection Map text file

In [10]:
#Removes whitespaces from the string
import re
import pandas as pd
def removeWhiteSpace(textToParse):
    return "".join(textToParse.split());


def parseNodeTimeObject(nodeText):
    """Takes in an individual node text and separates them into the Node-ID and infection time"""
    nodeText = removeWhiteSpace(nodeText);
    x = re.split("\(", nodeText);
    nodeID = x[0];
    time = re.split("\)", x[1])[0];
    return (nodeID, time);

def parseInfectedNodesList(textToParse):
    """Parses a list of infected nodes text and returns a list of nodes as a Python list"""
    textToParse = removeWhiteSpace(textToParse);
    infectedNodes = re.split(',', re.split('\]$', re.split('^\[', textToParse)[1])[0]);
    return infectedNodes;

attributeText1 = "infectionTime"
lst=[]
cluster=1
clst=[]
with open('Data/output1a-infectionMap.txt') as f:
    #read_data = f.read()
    lines = f.readlines()
    #print("file", f)
    for line in lines:
        if(line == '\n'):
            cluster+=1
            pass;
            #print ("This is an empty line!");
        else:
            #print(line);
            # First check how many "->" this infection line has.
            infectionTimes = []
            tempSplit = re.split("->", line)
            color="#FF0000"
            if len(tempSplit) == 2:
                #This is the initial phase in the chain
                tempSourceNode = parseNodeTimeObject(tempSplit[0]);
                tempTargetNodes = parseInfectedNodesList(tempSplit[1]);
                for nodeText in tempTargetNodes:
                    tempTargetNode = parseNodeTimeObject(nodeText);
                    temp = (tempSourceNode[0],tempTargetNode[0], cluster, color)
                    lst.append(temp)
                    #source = (tempSourceNode[0],cluster)
                    #clst.append(source)
                    #target = (tempTargetNode[0],cluster)
                    #clst.append(target)

            else:
                #This is further down the chain, if we are here, the source node should be already in the graph.
                tempSourceNode = parseNodeTimeObject(tempSplit[1]);
                tempTargetNodes = parseInfectedNodesList(tempSplit[2]);
                color=""
                for nodeText in tempTargetNodes:
                    tempTargetNode = parseNodeTimeObject(nodeText);
                    #print("Adding TargetNode:", tempTargetNode);
                    temp = (tempSourceNode[0], tempTargetNode[0], cluster,color)
                    lst.append(temp)
                    source = (tempSourceNode[0],cluster)
                    clst.append(source)
                    target = (tempTargetNode[0],cluster)
                    clst.append(target)


                    
infection_cluster = pd.DataFrame(lst,columns =['Source','Target', 'cluster','color'])
infection_cluster = pd.DataFrame.drop_duplicates(infection_cluster)
infection_cluster['Rank'] = infection_cluster.groupby(['cluster'])['cluster'].transform('count').rank(ascending=False, method='dense')

clusterrange=50
infection_cluster = infection_cluster[infection_cluster.Rank<=clusterrange]
infection_cluster = infection_cluster.applymap(str)
infection_cluster
infection_cluster.to_csv("infectionccccc.csv")
#infectionmap = pd.DataFrame(lst,columns =['Source', 'Target'])
#infectionmap = pd.DataFrame.drop_duplicates(infectionmap)


#infectionmap

#infection_cluster = pd.DataFrame(clst,columns =['Node', 'cluster'])
#infection_cluster = pd.DataFrame.drop_duplicates(infection_cluster)
#infection_cluster = infection_cluster.applymap(str)
#infection_cluster

In [11]:
infection_cluster

,Source,Target,cluster,color,Rank
168,176,2651,18,#FF0000,35.0
169,176,9910,18,#FF0000,35.0
170,176,16143,18,#FF0000,35.0
171,176,6877,18,#FF0000,35.0
172,176,7359,18,#FF0000,35.0
...,...,...,...,...,...
17911,19652,4955,1567,,14.0
17912,19652,4236,1567,,14.0
17913,17230,16664,1567,,14.0
17914,628,737,1567,,14.0


In [12]:
#Source Node and Cluster
SNC=infection_cluster[['Source','Rank','color']]
SNC = SNC.rename(columns={'Source':'Node'})
SNC = pd.DataFrame.drop_duplicates(SNC)
SNC.to_csv("infectionccccc.csv")
SNC

,Node,Rank,color
168,176,35.0,#FF0000
179,2651,35.0,
180,3590,35.0,
182,9910,35.0,
184,118,35.0,
...,...,...,...
17906,9662,14.0,
17907,19652,14.0,
17913,17230,14.0,
17914,628,14.0,


In [13]:
#Target Node and cluster
TNC=infection_cluster[['Target','Rank']]
TNC = TNC.rename(columns={'Target':'Node'})
TNC = pd.DataFrame.drop_duplicates(TNC)
TNC

,Node,Rank
168,2651,35.0
169,9910,35.0
170,16143,35.0
171,6877,35.0
172,7359,35.0
...,...,...
17911,4955,14.0
17912,4236,14.0
17913,16664,14.0
17914,737,14.0


In [14]:
#Combine Node and cluster
NC = SNC.append(TNC)
NC = NC.drop_duplicates(subset=['Node', 'Rank'], keep='first')
NC.to_csv("infectionccccc.csv")
NC

,Node,Rank,color
168,176,35.0,#FF0000
179,2651,35.0,
180,3590,35.0,
182,9910,35.0,
184,118,35.0,
...,...,...,...
17911,4955,14.0,NaN
17912,4236,14.0,NaN
17913,16664,14.0,NaN
17914,737,14.0,NaN


In [15]:
#Combine Contact_Covid_Model Dataset and InfectionMap Dataset
Combine_data = pd.merge(NC, NL, on=['Node', 'Node'], how='inner')
Combine_data.to_csv("infectionccccc.csv")
Combine_data

,Node,Rank,color,Location,bgcolor,shape,Cluster
0,176,35.0,#FF0000,School,#ffffb3,Msquare,2
1,2651,35.0,,School,#ffffb3,Msquare,2
2,3590,35.0,,Restaurant,#8dd3c7,circle,1
3,9910,35.0,,Restaurant,#8dd3c7,circle,1
4,118,35.0,,School,#ffffb3,Msquare,2
...,...,...,...,...,...,...,...
6720,4955,14.0,NaN,Restaurant,#8dd3c7,circle,1
6721,4236,14.0,NaN,Restaurant,#8dd3c7,circle,1
6722,16664,14.0,NaN,Restaurant,#8dd3c7,circle,1
6723,737,14.0,NaN,Restaurant,#8dd3c7,circle,1


In [16]:
#Primary Nodes for cluster Map
PN = Combine_data[Combine_data.color.isin(['#FF0000'])]
PN = PN[['Node','Rank','shape','color','Cluster','bgcolor']]
PN = pd.DataFrame.drop_duplicates(PN)
PN

,Node,Rank,shape,color,Cluster,bgcolor
0,176,35.0,Msquare,#FF0000,2,#ffffb3
16,345,45.0,circle,#FF0000,1,#8dd3c7
31,351,44.0,Msquare,#FF0000,2,#ffffb3
43,417,29.0,circle,#FF0000,1,#8dd3c7
64,419,33.0,triangle,#FF0000,5,#80b1d3
...,...,...,...,...,...,...
2477,19113,48.0,circle,#FF0000,1,#8dd3c7
2492,19242,31.0,circle,#FF0000,1,#8dd3c7
2512,19348,27.0,circle,#FF0000,1,#8dd3c7
2535,19522,38.0,circle,#FF0000,1,#8dd3c7


In [17]:
#Secondary Nodes for cluster
SN = Combine_data[~Combine_data.color.isin(['#FF0000'])]
SN = SN[['Node','Rank','shape','bgcolor','Cluster']]
SN = pd.DataFrame.drop_duplicates(SN)
SN

,Node,Rank,shape,bgcolor,Cluster
1,2651,35.0,Msquare,#ffffb3,2
2,3590,35.0,circle,#8dd3c7,1
3,9910,35.0,circle,#8dd3c7,1
4,118,35.0,Msquare,#ffffb3,2
5,16143,35.0,circle,#8dd3c7,1
...,...,...,...,...,...
6720,4955,14.0,circle,#8dd3c7,1
6721,4236,14.0,circle,#8dd3c7,1
6722,16664,14.0,circle,#8dd3c7,1
6723,737,14.0,circle,#8dd3c7,1


Combining infection location and assigning color

In [18]:
#secondary_node_color = pd.merge(event_infect, S, on=['Node', 'Node'], how='inner')
#secondary_node_color

In [19]:
#Dotfile Graph Preparation
#Cluster Description
graph = pd.DataFrame(['graph{'],columns=['dotfilegraphformat'])
graph

,dotfilegraphformat
0,graph{


In [20]:
#Cluster Description for primary Source Node
PNC = PN['Node'] +'[cluster='+ PN['Rank']+' ,shape="'+ PN['shape']+'" ,style="filled,rounded",' +' color="#0000FF"' +', fontname="Arial Bold", fontcolor="#FFFFFF"];'
PNC = pd.DataFrame(PNC,columns=['dotfilegraphformat'])
PNC = PNC.replace()
PNC

,dotfilegraphformat
0,"176[cluster=35.0 ,shape=""Msquare"" ,style=""fill..."
16,"345[cluster=45.0 ,shape=""circle"" ,style=""fille..."
31,"351[cluster=44.0 ,shape=""Msquare"" ,style=""fill..."
43,"417[cluster=29.0 ,shape=""circle"" ,style=""fille..."
64,"419[cluster=33.0 ,shape=""triangle"" ,style=""fil..."
...,...
2477,"19113[cluster=48.0 ,shape=""circle"" ,style=""fil..."
2492,"19242[cluster=31.0 ,shape=""circle"" ,style=""fil..."
2512,"19348[cluster=27.0 ,shape=""circle"" ,style=""fil..."
2535,"19522[cluster=38.0 ,shape=""circle"" ,style=""fil..."


In [21]:
graph = graph.append(PNC)
graph

,dotfilegraphformat
0,graph{
0,"176[cluster=35.0 ,shape=""Msquare"" ,style=""fill..."
16,"345[cluster=45.0 ,shape=""circle"" ,style=""fille..."
31,"351[cluster=44.0 ,shape=""Msquare"" ,style=""fill..."
43,"417[cluster=29.0 ,shape=""circle"" ,style=""fille..."
...,...
2477,"19113[cluster=48.0 ,shape=""circle"" ,style=""fil..."
2492,"19242[cluster=31.0 ,shape=""circle"" ,style=""fil..."
2512,"19348[cluster=27.0 ,shape=""circle"" ,style=""fil..."
2535,"19522[cluster=38.0 ,shape=""circle"" ,style=""fil..."


In [22]:
#Cluster Description for secondary Source Node
SNC = SN['Node'] +'[cluster='+ SN['Rank']+', shape="'+ SN['shape']+'", style="filled", '+'color="'+ SN['bgcolor']+'"];'
SNC = SNC.replace()
SNC = pd.DataFrame(SNC,columns=['dotfilegraphformat'])
SNC

,dotfilegraphformat
1,"2651[cluster=35.0, shape=""Msquare"", style=""fil..."
2,"3590[cluster=35.0, shape=""circle"", style=""fill..."
3,"9910[cluster=35.0, shape=""circle"", style=""fill..."
4,"118[cluster=35.0, shape=""Msquare"", style=""fill..."
5,"16143[cluster=35.0, shape=""circle"", style=""fil..."
...,...
6720,"4955[cluster=14.0, shape=""circle"", style=""fill..."
6721,"4236[cluster=14.0, shape=""circle"", style=""fill..."
6722,"16664[cluster=14.0, shape=""circle"", style=""fil..."
6723,"737[cluster=14.0, shape=""circle"", style=""fille..."


In [23]:
graph = graph.append(SNC)
graph

,dotfilegraphformat
0,graph{
0,"176[cluster=35.0 ,shape=""Msquare"" ,style=""fill..."
16,"345[cluster=45.0 ,shape=""circle"" ,style=""fille..."
31,"351[cluster=44.0 ,shape=""Msquare"" ,style=""fill..."
43,"417[cluster=29.0 ,shape=""circle"" ,style=""fille..."
...,...
6720,"4955[cluster=14.0, shape=""circle"", style=""fill..."
6721,"4236[cluster=14.0, shape=""circle"", style=""fill..."
6722,"16664[cluster=14.0, shape=""circle"", style=""fil..."
6723,"737[cluster=14.0, shape=""circle"", style=""fille..."


In [24]:
#source and target infected nodes
IC = infection_cluster['Source'] +'--'+ infection_cluster['Target']+';'
IC = pd.DataFrame(IC,columns=['dotfilegraphformat'])
IC

,dotfilegraphformat
168,176--2651;
169,176--9910;
170,176--16143;
171,176--6877;
172,176--7359;
...,...
17911,19652--4955;
17912,19652--4236;
17913,17230--16664;
17914,628--737;


In [25]:
graph  = graph.append(IC)
graph

,dotfilegraphformat
0,graph{
0,"176[cluster=35.0 ,shape=""Msquare"" ,style=""fill..."
16,"345[cluster=45.0 ,shape=""circle"" ,style=""fille..."
31,"351[cluster=44.0 ,shape=""Msquare"" ,style=""fill..."
43,"417[cluster=29.0 ,shape=""circle"" ,style=""fille..."
...,...
17911,19652--4955;
17912,19652--4236;
17913,17230--16664;
17914,628--737;


In [26]:
graph = graph.append(pd.DataFrame(['}'],columns=['dotfilegraphformat']))
graph

,dotfilegraphformat
0,graph{
0,"176[cluster=35.0 ,shape=""Msquare"" ,style=""fill..."
16,"345[cluster=45.0 ,shape=""circle"" ,style=""fille..."
31,"351[cluster=44.0 ,shape=""Msquare"" ,style=""fill..."
43,"417[cluster=29.0 ,shape=""circle"" ,style=""fille..."
...,...
17912,19652--4236;
17913,17230--16664;
17914,628--737;
17915,9857--9859;


In [27]:
numpy_array = graph.to_numpy()
np.savetxt("infection_connected_cluster.gv", numpy_array, fmt="%s")

In [28]:
#p.render('infection_cluster.gv', view=True)  # doctest: +SKIP
#'test-output/round-table.gv.pdf'

For the Location wise gmap

In [29]:
#Dotfile Graph Preparation
#Cluster Description
Lgraph = pd.DataFrame(['graph{'],columns=['dotfilegraphformat'])
Lgraph

,dotfilegraphformat
0,graph{


In [30]:
#Cluster Description for primary Source Node
LPNC = PN['Node'] +'[cluster='+ PN['Cluster']+', clustercolor="'+ PN['bgcolor']+'", shape="'+ PN['shape']+'", style="filled",' +' color="BLUE"' +', fontname="Arial Bold", fontcolor="#FFFFFF"];'
LPNC = pd.DataFrame(LPNC,columns=['dotfilegraphformat'])
LPNC = LPNC.replace()
LPNC.to_csv("infectionccccc.csv")
LPNC


,dotfilegraphformat
0,"176[cluster=2, clustercolor=""#ffffb3"", shape=""..."
16,"345[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
31,"351[cluster=2, clustercolor=""#ffffb3"", shape=""..."
43,"417[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
64,"419[cluster=5, clustercolor=""#80b1d3"", shape=""..."
...,...
2477,"19113[cluster=1, clustercolor=""#8dd3c7"", shape..."
2492,"19242[cluster=1, clustercolor=""#8dd3c7"", shape..."
2512,"19348[cluster=1, clustercolor=""#8dd3c7"", shape..."
2535,"19522[cluster=1, clustercolor=""#8dd3c7"", shape..."


In [31]:
Lgraph = Lgraph.append(LPNC)
Lgraph

,dotfilegraphformat
0,graph{
0,"176[cluster=2, clustercolor=""#ffffb3"", shape=""..."
16,"345[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
31,"351[cluster=2, clustercolor=""#ffffb3"", shape=""..."
43,"417[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
...,...
2477,"19113[cluster=1, clustercolor=""#8dd3c7"", shape..."
2492,"19242[cluster=1, clustercolor=""#8dd3c7"", shape..."
2512,"19348[cluster=1, clustercolor=""#8dd3c7"", shape..."
2535,"19522[cluster=1, clustercolor=""#8dd3c7"", shape..."


In [32]:
#Cluster Description for secondary Source Node
LSNC = SN['Node'] +'[cluster='+ SN['Cluster']+', clustercolor="'+ SN['bgcolor']+'", shape="'+ SN['shape']+'"];'#, style="filled'+'"];'
LSNC = LSNC.replace()
LSNC = pd.DataFrame(LSNC,columns=['dotfilegraphformat'])
LSNC

,dotfilegraphformat
1,"2651[cluster=2, clustercolor=""#ffffb3"", shape=..."
2,"3590[cluster=1, clustercolor=""#8dd3c7"", shape=..."
3,"9910[cluster=1, clustercolor=""#8dd3c7"", shape=..."
4,"118[cluster=2, clustercolor=""#ffffb3"", shape=""..."
5,"16143[cluster=1, clustercolor=""#8dd3c7"", shape..."
...,...
6720,"4955[cluster=1, clustercolor=""#8dd3c7"", shape=..."
6721,"4236[cluster=1, clustercolor=""#8dd3c7"", shape=..."
6722,"16664[cluster=1, clustercolor=""#8dd3c7"", shape..."
6723,"737[cluster=1, clustercolor=""#8dd3c7"", shape=""..."


In [33]:
Lgraph = Lgraph.append(LSNC)
Lgraph

,dotfilegraphformat
0,graph{
0,"176[cluster=2, clustercolor=""#ffffb3"", shape=""..."
16,"345[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
31,"351[cluster=2, clustercolor=""#ffffb3"", shape=""..."
43,"417[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
...,...
6720,"4955[cluster=1, clustercolor=""#8dd3c7"", shape=..."
6721,"4236[cluster=1, clustercolor=""#8dd3c7"", shape=..."
6722,"16664[cluster=1, clustercolor=""#8dd3c7"", shape..."
6723,"737[cluster=1, clustercolor=""#8dd3c7"", shape=""..."


In [34]:
#source and target infected nodes
LIC = infection_cluster['Source'] +'--'+ infection_cluster['Target']+';'
LIC = pd.DataFrame(LIC,columns=['dotfilegraphformat'])
LIC

,dotfilegraphformat
168,176--2651;
169,176--9910;
170,176--16143;
171,176--6877;
172,176--7359;
...,...
17911,19652--4955;
17912,19652--4236;
17913,17230--16664;
17914,628--737;


In [35]:
Lgraph  = Lgraph.append(LIC)
Lgraph

,dotfilegraphformat
0,graph{
0,"176[cluster=2, clustercolor=""#ffffb3"", shape=""..."
16,"345[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
31,"351[cluster=2, clustercolor=""#ffffb3"", shape=""..."
43,"417[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
...,...
17911,19652--4955;
17912,19652--4236;
17913,17230--16664;
17914,628--737;


In [36]:
Lgraph = Lgraph.append(pd.DataFrame(['}'],columns=['dotfilegraphformat']))
Lgraph

,dotfilegraphformat
0,graph{
0,"176[cluster=2, clustercolor=""#ffffb3"", shape=""..."
16,"345[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
31,"351[cluster=2, clustercolor=""#ffffb3"", shape=""..."
43,"417[cluster=1, clustercolor=""#8dd3c7"", shape=""..."
...,...
17912,19652--4236;
17913,17230--16664;
17914,628--737;
17915,9857--9859;


In [37]:
Lnumpy_array = Lgraph.to_numpy()
np.savetxt("infection_Location_cluster.gv", Lnumpy_array, fmt="%s")

In [38]:
#p.render('infection_cluster.gv', view=True)  # doctest: +SKIP
#'test-output/round-table.gv.pdf'